In [1]:
%load_ext rich
from pathlib import Path
from corpus_base import settings, DecisionRow

In [2]:
from corpus_base import init
init()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/7g/qc77bj6j63q0gr0mrzvbmxkc0000gn/T/ipykernel_71089/2356359417.py:2 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/7g/qc77bj6j63q0gr0mrzvbmxkc0000gn/T/ipykernel_71089/2356359417.py'                 │
│                                                                                                  │
│ /Users/mv/Code/corpus-base/corpus_base/__main__.py:75 in init                                    │
│                                                                                                  │
│   72 │                                                                                           │
│   73 │   # insert justices into the justice table                                                │
│   74 │   Justice.from_api()                                                                      │
│ ❱ 75 │   Justice.init_justices_tbl()                                                             │
│   76 │                                                                                           │
│   77 │   # infuse decision tables from path                                                      │
│   78 │   for counter, details_file in enumerate(settings.case_folders):                          │
│                                                                                                  │
│ /Users/mv/Code/corpus-base/corpus_base/justice.py:222 in init_justices_tbl                       │
│                                                                                                  │
│   219 │   @classmethod                                                                           │
│   220 │   def init_justices_tbl(cls):                                                            │
│   221 │   │   """Add a table containing names and ids of justices; alter the original decision   │
│ ❱ 222 │   │   return just_tbl.insert_all(i.dict() for i in Justice.from_local())                 │
│   223 │                                                                                          │
│   224 │   @classmethod                                                                           │
│   225 │   def get_active_on_date(cls, target_date: str) -> list[dict]:                           │
│                                                                                                  │
│ /Users/mv/Code/corpus-base/.venv/lib/python3.10/site-packages/sqlite_utils/db.py:3093 in         │
│ insert_all                                                                                       │
│                                                                                                  │
│   3090 │   │   │                                                                                 │
│   3091 │   │   │   first = False                                                                 │
│   3092 │   │   │                                                                                 │
│ ❱ 3093 │   │   │   self.insert_chunk(                                                            │
│   3094 │   │   │   │   alter,                                                                    │
│   3095 │   │   │   │   extracts,                                                                 │
│   3096 │   │   │   │   chunk,                                                                    │
│                                                                                                  │
│ /Users/mv/Code/corpus-base/.venv/lib/python3.10/site-packages/sqlite_utils/db.py:2858 in         │
│ insert_chunk                                                                                     │
│                                                            

In [ ]:
f = next(Path().home().joinpath(settings.DecisionSourceFiles).glob("**/50983/"))

In [ ]:
obj = DecisionRow.from_path(f / "details.yaml")

In [ ]:
from pathlib import Path
from corpus_base import settings, DecisionRow
p = Path().cwd() / "tests" / "decisions" / "sc" / "62206" / "details.yaml"
obj = DecisionRow.from_path(p)